# <center>企业级大模型部署推理管理工具</center>

## <center>Part 1. Vllm 框架基础入门与本地私有化部署</center>

&emsp;&emsp;对开源大模型来说，虽然模型权重开源，但并不意味着这样的模型就可以开箱即用，而是需要一些框架来支撑其运行和推理。无论哪个公司旗下的模型，现在都在兼容同一个接入规范，即`OpenAI`兼容接口，因为这种`API`规范几乎可以与任何`SDK`或者客户端无缝集成，所以对大模型部署框架而言，是否兼容`OpenAI`规范也就成为了衡量一个其是否成熟、是否可以被广泛使用的重要指标。所以大家都关注的问题就是：部署与`OpenAI API`规范兼容模型的最佳框架是什么？

&emsp;&emsp;如下所示，目前主流的大模型部署框架以`llama.cpp`、`Ollama`和`Vllm`为主，其各自项目在`Github`上的`Stars`增长曲线如下所示。其中`Ollama`的`Stars`增长曲线是断档式领先，而`llama.cpp`和`Vllm`的`Stars`增长曲线则相对平缓，处于一个稳步增长的趋势。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181034368.png" width=60%></div>

&emsp;&emsp;之所以会呈现出这种现状，其实与各个框架的定位有本质的区别。首先，`llama.cpp` 是使用没有任何依赖关系的纯 C/C++ 实现，性能很高，可定制化和优化项非常多，但也因为底层是`C`语言，直接导致上手难度极高。`Ollama` 之所以能够受到最多开发者的关注，一个最根本的原因就是其部署和使用太简单了，兼容多种操作系统，且都提供了一键安装、单命令启动的快捷方式，可以极大降低初学者使用开源大模型的门槛，同时`Ollama`框架提供原生 `REST API` 和 `OpenAI API` 兼容性，也可以非常轻松的接入到其他的客户端中。

# 1. Vllm 框架整体概览

&emsp;&emsp;如果说`Ollama`的优势在于其简洁性，那么`Vllm`则是一个另辟蹊径、优先考虑性能和可扩展性的大模型部署框架，其核心的优化点在`高效内存管理`、`持续批处理功能`和`张量并行性`，从而在生产环境中的高吞吐量场景中表现极佳，同时这也是为什么`Vllm`框架是目前最适用于企业真实生产环境部署的根本原因。

&emsp;&emsp;`Vllm` 底层是基于`Pytorch` 构建，其`Gtihub` 开源地址为：https://github.com/vllm-project/vllm 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181137533.png" width=60%></div>

&emsp;&emsp;从各种基准测试数据来看，同等配置下，使用 `vLLM` 框架与 `Transformer` 等传统推理库相比，其吞吐量可以提高一个数量级，这归功于以下几个特性：

- **高级 GPU 优化**：利用 `CUDA` 和 `PyTorch` 最大限度地提高 `GPU` 利用率，从而实现更快的推理速度。`Ollama`其实是对`CPU-GPU`的混合应用，但`vllm`是针对纯`GPU`的优化。
- **高级内存管理**：通过`PagedAttention`算法实现对 `KV cache`的高效管理，减少内存浪费，从而优化大模型的运行效率。
- **批处理功能**：支持连续批处理和异步处理，从而提高多个并发请求的吞吐量。
- **安全特性**：内置 `API` 密钥支持和适当的请求验证，不像其他完全跳过身份验证的框架。
- **易用性**：`vLLM` 与 `HuggingFace` 模型无缝集成，支持多种流行的大型语言模型，并兼容 `OpenAI` 的 `API` 服务器。

&emsp;&emsp;以上核心的优化点原理以及应用方法，我们将在接下来的课程进行详细的讲解和实践。本节课程我们需要先进行`Vllm` 框架的私有化部署的学习。

&emsp;&emsp; `vllm` 框架主要支持纯文本和多模态两种模式的模型，其中对支持的纯文本类语言模型可以在这里看到：https://docs.vllm.ai/en/latest/models/supported_models.html#supported-text-models

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511604.png" width=60%></div>

&emsp;&emsp;其次是多模态模型，`Vllm`框架目前已经支持文本、图片、视频和音频四种模态的输入输出格式兼容规范，但是多模态模型官方并没有给出具体的模型列表，所以这里我们要进入到源码中对模型的集成文件中去匹配：https://github.com/vllm-project/vllm/tree/main/vllm/model_executor/models

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181304597.png" width=60%></div>

&emsp;&emsp;根据各个模型支持`.py`文件中的代码细节，这里梳理了一份目前最新版本`Vllm 0.8.4`支持的多模态模型如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>多模态模型支持列表</font></p>
<div class="center">

| .py 文件                          | 模型名称                          | 描述                                       |
|----------------------------------|----------------------------------|------------------------------------------|
| aya_vision.py                    | aya-vision                         | https://huggingface.co/CohereLabs/aya-vision-8b       |
| blip.py / blip2.py              | BLIP                            | 经典视觉-语言模型（BLIP 系列）。               |
| clip.py                          | CLIP                            | 经典的多模态对比学习模型（CLIP）。             |
| deepseek_vl2.py                  | DeepSeek                       | DeepSeek 的多模态版本（VL 表示 Vision-Language）。 |
| florence2.py                    | 微软模型                        | 微软的多模态模型。https://huggingface.co/microsoft/Florence-2-large                           |
| fuyu.py                          | Fuyu                           | Adept 的多模态模型（Fuyu 系列）。              |
| gemma3_mm.py                     | Gemma                          | mm 表示 Multimodal（Gemma 的多模态扩展）。    |
| glm4v.py                         | GLM-4                          | GLM-4 的多模态版本（v 表示 Vision）。         |
| h2ovl.py                         | H2O                            | H2O 的视觉-语言模型（vl 后缀）。              |
| idefics2_vision_model.py / idefics3.py | Idefics                     | Meta 的多模态模型（Idefics 系列）。       |
| internvl.py                      | InternVL-Chat                   | 上海 AI Lab 的多模态模型。                   |
| kimi_vl.py                       | Kimi-VL-A3B                        | 新增的 Kimi-VL 模型  |
| llava.py / llava_next.py / llava_next_video.py / llava_onevision.py | LLaVA                        | LLaVA 系列及其变种（开源视觉-语言模型）。 |
| mllama4.py                       | LLaMA-4                       | 多模态版本的 LLaMA-4。                      |
| molmo.py                         | Molmo                     | 提交记录涉及多模态数据处理。                   |
| moonvit.py                       | Kimi-VL                        | 与 Kimi-VL 同时新增的多模态模型。             |
| nvlm_d.py                        | NVLM-D-72B                         | NVIDIA 的多模态模型（vl 相关）。              |
| paligemma.py                     | PaLI-Gemma                     | Google 的 PaLI-Gemma 多模态模型。            |
| phi3v.py / phi4mm.py / phi4mm_audio.py | Phi-3 / Phi-4               | Phi-3 和 Phi-4 的多模态版本（支持视觉或音频）。 |
| prithvi_geospatial_mae.py        | Prithvi-EO-1.0                    | 地理空间多模态模型（NASA 合作项目）。         |
| qwen2_5_vl.py / qwen2_audio.py / qwen2_vl.py / qwen_vl.py | 通义千问                     | 通义千问的多模态版本（视觉或音频）。      |
| siglip.py                        | SigLIP                        | Google 的 SigLIP（多模态对比学习）。         |
| skyworkr1v.py                    | Skywork-R1V-38B                       | 幻方多模态模型（v 后缀）。                   |
| smolvlm.py                       | SmolVLM                     | 轻量级多模态模型（新增支持）。     |
| whisper.py                       | whisper-large-v3                   | OpenAI 的语音-文本多模态模型（虽然主要处理音频，但属于跨模态）。 |
</div>

# 2. Linux 操作系统部署 Vllm

&emsp;&emsp; 首先需要重点说明的是：<font color="red">`Vllm` 框架仅支持`Linux` 操作系统，官方并没有提供`Windows`的兼容版本</font>。同时除了有操作的限制，<font color="red">对运行的`Python`版本也要求在`Python 3.8` ~ `Python 3.12` 之间</font>。这两个条件限制为部署`Vllm`的先决条件，即必须满足才可以顺利使用`Vllm`启动大模型并提供推理服务。

&emsp;&emsp;这个策略非常符合企业级的部署策略，`Vllm` 框架作为目前在实际生产环境中使用最为广泛的框架之一，其对`Linux` 操作系统的支持，以及对`Python` 版本的限制，都是为了确保在生产环境中能够稳定运行，并且能够提供高性能的推理服务。（在生产环境中，没有任何一家企业会在`Windows` 操作系统上部署模型服务或者应用服务）

&emsp;&emsp;因此，我们接下来就严格按照操作系统和`Python` 版本的要求，来进行`Vllm` 框架部署和使用的详细讲解和实践。


&emsp;&emsp; 这里我们采用的`Linux` 操作系统是`Ubuntu 22.04`，同时配置四张`3090` 显卡来进行`Vllm` 框架的部署和使用。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511611.png" width=80%></div>

&emsp;&emsp; `Vllm` 工具已经通过编译并上传到`PYPI` 仓库以及主流的包管理平台中，所以可以直接使用如`pip`、`conda` 等工具进行快速安装。这里我们使用比较常用的`conda` 包版本管理工具进行安装部署。


- **Step 1. 安装`conda` 包版本管理工具**

&emsp;&emsp;首先需要检查当前使用的操作系统是否已经安装了`conda` 包版本管理工具，可以通过`conda --version` 命令查看`Conda` 版本，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171523767.png" width=80%></div>

&emsp;&emsp;如果出现`Conda not found` 等报错，需要先安装`Conda` 环境，再执行接下来的步骤。


- **Step 2. 创建Python虚拟环境**

&emsp;&emsp;`vllm`官方要求的是`Python 3.9` ~ `Python 3.12` 之间的版本，这里我们选择`Python 3.12` 版本。执行如下命令进行创建：

```bash
    conda create --name vllm python=3.12
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511612.png" width=80%></div>

- **Step 3. 激活虚拟环境**


&emsp;&emsp;创建完虚拟环境后，使用`Conda` 激活虚拟环境，通过`conda activate vllm` 命令激活，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511613.png" width=80%></div>

&emsp;&emsp;接下来，使用`pip` 安装`vllm` 框架，执行如下命令：

```bash
    pip install vllm
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511614.png" width=80%></div>

&emsp;&emsp;此时耐心等待安装完成即可。待安装完成后，可以使用`pip show vllm` 命令查看`vllm` 框架的安装信息，可以明确看到当前安装的`vllm` 版本号。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511615.png" width=80%></div>

&emsp;&emsp;至此，`Vllm` 框架的安装就已经完成了。整个过程就是非常常规的依赖包安装过程，唯一需要注意的就是`Python` 版本的要求。

&emsp;&emsp;在安装完成`Vllm` 框架后，就可以开始进行大模型推理服务的启动和调用了。`Vllm` 框架提供了两种启动并调用大模型生成推理服务的方式，分别是<font color="red">离线推理</font>和<font color="red">在线推理</font>。其中：

- **离线推理**： 离线推理类似于使用 `Pytorch` 模块一样，当我们需要使用大模型生成推理服务时，先加载模型，然后使用输入数据运行该模型，并获取输出结果。
- **在线推理**： 在线推理类似于有一个服务器，可以先启动大模型，然后等待来自客户端的请求，一旦接收到请求，就会使用大模型生成推理服务，并返回结果，并且可以同时处理多个请求。

&emsp;&emsp;这是两种不同推理方式最本质的区别，毫无疑问<font color="red">在线推理是更加符合实际生产环境的</font>。但为了帮助大家更好的理解`Vllm` 框架，这里我们还是先从离线推理开始讲解，然后再重点讲解在线推理。


# 3. vllm 离线推理

&emsp;&emsp;无论是在线推理还是离线推理，都采用的是`API` 接口的方式来调用大模型生成推理服务。对离线推理来说，其加载模型、传输数据、获取结果的`API` 调用形式如下所示：

```python
    from vllm import LLM
    # 加载模型
    llm = LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")
    
    # 传输数据
    outputs = llm.generate("你好，我是木羽")

    # 获取结果
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")
```



&emsp;&emsp;代码看起来简单，但如果想要跑通这几行代码，则需要我们提前做两件非常关键的准备工作：<font color="red">其一是需要在`Linux` 服务器上下载大模型权重，其二则是需要配置一个可以加载服务器虚拟环境（即上一步创建的`vllm` 虚拟环境）的`Python IDE`解释器</font>。


&emsp;&emsp;我们就以`DeepSeek-R1-Distill-Qwen-32B` 大模型为例，默认情况下，`LLM(model="deepseek-ai/DeepSeek-R1-Distill-Qwen-32B")` 这行命令的执行逻辑是：先检测当前服务器中是否存在`deepseek-ai/DeepSeek-R1-Distill-Qwen-32B` 的模型权重，如果存在则直接加载该目录下的模型权重，否则会从`Hugging Face` 上下载大模型权重。其存在的主要问题是：国内的服务器是没有办法访问`Hugging Face` 网站的，所以一定会报错。因此，国内的开发者需要采用更有效的方式来确保大模型权重的正确下载，即使用国内镜像源`ModelScope` 来执行模型权重的本地化存储。

&emsp;&emsp;回到`Linux` 服务器中，首先通过如下命令安装`ModelScope` 的`pip` 包：

```bash
    pip install modelscope
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511620.png" width=80%></div>

&emsp;&emsp;接下来，进入到`ModelScope` 的主页中：https://www.modelscope.cn/home

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613863.png" width=80%></div>


&emsp;&emsp;通过关键词找到对应的模型详情页，比如我这里以`DeepSeek-R1-Distill-Qwen-32B` 为例，点击进入详情页，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613865.png" width=80%></div>

&emsp;&emsp;进入到模型详情页后，点击`模型文件`，然后再找到`下载模型`按钮，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613866.png" width=80%></div>

&emsp;&emsp;在弹出来的会话框中，找到`SDK 下载` 标签，复制对应的模型服务指针标识：如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171613867.png" width=80%></div>

&emsp;&emsp;接下来回到`Linux` 服务器中，通过`vim` 新建一个`model_download.py` 文件：

```python
    vim model_download.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511621.png" width=80%></div>

&emsp;&emsp;然后，按`i` 键进入编辑模式，将复制的代码粘贴到文件中：

```python
    from modelscope import snapshot_download

    # model_dir = snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-32B', cache_dir='/root/autodl-tmp', revision='master')
    model_dir = snapshot_download('deepseek-ai/DeepSeek-R1-Distill-Qwen-32B', cache_dir='/home/08_vllm', revision='master')
```

&emsp;&emsp;注意，这里`cache_dir` 参数指定的路径是模型权重存储的路径，这里我们指定为`/home/08_vllm` 目录，即模型权重会存储到该目录下。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511622.png" width=80%></div>

&emsp;&emsp;修改完成后，按`Esc` 键退出编辑模式，再按`Shift+:` 键进入命令模式，输入`wq` 保存并退出。执行如下命令，运行`model_download.py` 文件：

```bash
    python model_download.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511623.png" width=80%></div>

&emsp;&emsp;耐心等待模型权重下载完成即可。待下载完成后，可以看到`/home/08_vllm` 目录下会多出一个`deepseek-ai` 文件夹，里面存放的就是`DeepSeek-R1-Distill-Qwen-32B` 大模型的权重文件。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171622553.png" width=80%></div>

&emsp;&emsp;按照上述流程中`model_download.py`下载模型的方式，可以允许我们随意下载任意在`ModelScope`中托管的模型权重至本地，因此这里依次下载了`Qwen2.5`系列和`DeepSeek R1` 系列的不同尺寸的模型，大家也可以根据自己的需求灵活选择模型。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181342835.png" width=80%></div>

&emsp;&emsp;至此，大模型权重就已经下载完成了。接下来，我们需要做的是配置一个可以加载服务器虚拟环境（即上一步创建的`vllm` 虚拟环境）的`Python IDF`解释器。这一步就会根据大家实际的开发情况产生不同的配置方法，总的来说会分成以下两种情况：

1. 部署大模型的服务器和你要执行代码调用的`Python IDE`是同一台服务器，则可以直接选择到对应的虚拟环境并执行代码；
2. 部署大模型的服务器和你要执行代码调用的`Python IDE`不是同一台服务器，则需要通过`SSH` 连接到部署大模型的服务器，然后选择到对应的虚拟环境并执行代码。

&emsp;&emsp;同一台服务器的情况非常简单，直接打开`Python IDE`，选择到对应的虚拟环境并执行代码即可。这里重点介绍第二种情况，即部署大模型的服务器和你要执行代码调用的`Python IDE`不是同一台服务器。同时这也是最常见的企业开发环境。


&emsp;&emsp;这类情况用最通俗的理解是：我们把大模型部署在了局域网/租赁的云服务器中，希望可以在本地电脑上执行代码调用模型服务。这类情况我们要做如下配置：（以`Jupyter Notebook`为例）:

&emsp;&emsp;首先进入在局域网/租赁的云服务器的`Linux` 服务器中，在`vllm` 虚拟环境中安装`ipykernel` 和`jupyter` 包，执行如下命令：
```bash
    pip install ipykernel jupyter
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511616.png" width=80%></div>


&emsp;&emsp;安全起见，对连接时的密码进行加密处理，否则明文写在配置文件中，容易造成数据安全风险，依次执行如下操作：
```bash
    from jupyter_server.auth import passwd
    passwd()
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753935.png" width=80%></div>

&emsp;&emsp;完成密码加密后，执行如下命令生成Jupyter Lab 的配置文件（jupyter_lab_config.py）：
```bash
    jupyter lab --generate-config
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753934.png" width=80%></div>

&emsp;&emsp;使用`Vim`编辑器，找到如下配置，执行修改：
```bash
    c.ServerApp.allow_origin = '*'
    c.ServerApp.allow_remote_access = True
    c.ServerApp.ip = '0.0.0.0'
    c.ServerApp.open_browser = False  
    c.ServerApp.password = '加密后的密码'（上一步复制的加密串）
    c.ServerApp.port = 8002 
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753936.png" width=80%></div>

&emsp;&emsp;全部配置完成后，在服务器端启动Jupyter Lab服务，通过如下命令后台启动：
```bash
    nohup jupyter lab --allow-root > jupyterlab.log 2>&1 &
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753937.png" width=80%></div>

&emsp;&emsp;然后，将虚拟环境的`Kernel`写入`Jupyter Lab`，执行如下命令，创建一个`ipykernel` 内核：
```bash
    python -m ipykernel install --user --name vllm --display-name "vllm"
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511617.png" width=80%></div>

&emsp;&emsp;接下来，在浏览器中打开`Jupyter Notebook` 服务，在`Jupyter kernel` 中就会出现选择`vllm` 内核的选项，如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511618.png" width=80%></div>

&emsp;&emsp;选择`vllm` 内核并创建一个新的`notebook` 笔记本，可以通过`! pip show vllm` 快速验证是否正常加载到了`vllm` 的虚拟环境：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504171511619.png" width=80%></div>

In [1]:
# ! pip show vllm

Name: vllm
Version: 0.8.4
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License: Apache License
                           Version 2.0, January 2004
                        http://www.apache.org/licenses/

   TERMS AND CONDITIONS FOR USE, REPRODUCTION, AND DISTRIBUTION

   1. Definitions.

      "License" shall mean the terms and conditions for use, reproduction,
      and distribution as defined by Sections 1 through 9 of this document.

      "Licensor" shall mean the copyright owner or entity authorized by
      the copyright owner that is granting the License.

      "Legal Entity" shall mean the union of the acting entity and all
      other entities that control, are controlled by, or are under common
      control with that entity. For the purposes of this definition,
      "control" means (i) the power, direct or indirect, to cause the
      direction or mana

&emsp;&emsp;如果正常加载到了`vllm` 的虚拟环境，则可以看到`vllm` 的版本号，这里可以看到，与我们在服务器上安装的版本号是一致的。说明我们已经在本地电脑上成功加载到了`vllm` 的虚拟环境。

&emsp;&emsp;接下来，我们可以通过离线推理的`API` 接口来调用大模型生成推理服务。我们会对对话模型和推理模型依次展开详细的测试。


&emsp;&emsp;首先以`Qwen2.5`系列为例测试在`vllm`框架下对对话类模型的使用。默认情况下，`Vllm` 从`huggingface`下载模型权重，并使用`transformers`库加载模型。这里我们的模型是从`ModelScope`下载的，所以在调用离线推理`API`时，需要搭配`trust_remote_code=True`参数，其次，对`model` 参数，指的是模型权重文件的本地路径，因此需要大家根据模型下载路径进行修改。即实例化代码如下图所示：

In [1]:
from vllm import LLM

llm = LLM(model="/home/08_vllm/qwen/Qwen2___5-7B",
          trust_remote_code=True,
          max_model_len=4096,
)

INFO 04-18 14:21:53 [__init__.py:239] Automatically detected platform cuda.
INFO 04-18 14:22:03 [config.py:689] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward', 'score'}. Defaulting to 'generate'.
INFO 04-18 14:22:03 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-18 14:22:04 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/home/08_vllm/qwen/Qwen2___5-7B', speculative_config=None, tokenizer='/home/08_vllm/qwen/Qwen2___5-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 04-18 14:22:10 [loader.py:458] Loading weights took 4.76 seconds
INFO 04-18 14:22:11 [gpu_model_runner.py:1291] Model loading took 14.2717 GiB and 4.991946 seconds
INFO 04-18 14:22:21 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/f2ded477b0/rank_0_0 for vLLM's torch.compile
INFO 04-18 14:22:21 [backends.py:426] Dynamo bytecode transform time: 10.11 s
INFO 04-18 14:22:22 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 04-18 14:22:28 [monitor.py:33] torch.compile takes 10.11 s in total
INFO 04-18 14:22:31 [kv_cache_utils.py:634] GPU KV cache size: 97,680 tokens
INFO 04-18 14:22:31 [kv_cache_utils.py:637] Maximum concurrency for 4,096 tokens per request: 23.85x
INFO 04-18 14:22:55 [gpu_model_runner.py:1626] Graph capturing finished in 24 secs, took 1.44 GiB
INFO 04-18 14:22:55 [core.py:163] init engine (profile, create kv cache, warmup model) took 43.92 seconds
INFO 04-18 14:22:55 [core_client.py:435] Core engine 

&emsp;&emsp;`LLM` 类是运行离线推理的主要类。当使用`LLM`类进行模型初始化的加载时，有如下几个最初级且重要的参数需要大家掌握：

1. **device**: 该参数会指定模型加载的设备，vllm 支持在 `cuda, neuron, cpu, tpu, xpu, hpu`，默认是`auto`, 即初始化时会自动识别当前系统中可用的设备。
2. **tensor_paralleis***ze"：该参数会指定模型可以通过`GPU`的`Tensor`并行运行，默认是`1`，即不使用`GPU`的`Tensor`并行，但如果加载的模型参数超过单个`GPU`的显存，即使服务器上有多个`GPU`，也会报错`Out of memory`，并不会自动使用其他`GPU`进行并行。
3. **gpu_memory_utilization**：该参数会指定模型加载时，`GPU`的显存取值范围为`0~1`，使用率，默认是`0.90`，即`GPU`的显存使用率不会0过`95%`，但该参数会直接占用`GPU`的显存，以避免`GPU`的显存被其他进占用。


&emsp;&emsp;比如对`Qwen2.5-7B`模型，采用默认配置其显存占用情况如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181430516.png" width=80%></div>

&emsp;&emsp;如果模型较大，则可以指定多块`GPU`，比如如下所示的参数调整：

In [2]:
from vllm import LLM

llm = LLM(model="/home/08_vllm/qwen/Qwen2___5-7B",
          trust_remote_code=True,
          tensor_parallel_size=2,
          gpu_memory_utilization=0.8,
          max_model_len=4096,
)

INFO 04-18 14:33:04 [__init__.py:239] Automatically detected platform cuda.
INFO 04-18 14:33:13 [config.py:689] This model supports multiple tasks: {'embed', 'reward', 'generate', 'classify', 'score'}. Defaulting to 'generate'.
INFO 04-18 14:33:13 [config.py:1713] Defaulting to use mp for distributed inference
INFO 04-18 14:33:13 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-18 14:33:15 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/home/08_vllm/qwen/Qwen2___5-7B', speculative_config=None, tokenizer='/home/08_vllm/qwen/Qwen2___5-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, d

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=182914) INFO 04-18 14:33:22 [loader.py:458] Loading weights took 4.17 seconds
(VllmWorker rank=0 pid=182840) INFO 04-18 14:33:22 [loader.py:458] Loading weights took 4.19 seconds
(VllmWorker rank=1 pid=182914) INFO 04-18 14:33:23 [gpu_model_runner.py:1291] Model loading took 7.1441 GiB and 4.411589 seconds
(VllmWorker rank=0 pid=182840) INFO 04-18 14:33:23 [gpu_model_runner.py:1291] Model loading took 7.1441 GiB and 4.425101 seconds
(VllmWorker rank=0 pid=182840) INFO 04-18 14:33:34 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/b6232da3fe/rank_0_0 for vLLM's torch.compile
(VllmWorker rank=1 pid=182914) INFO 04-18 14:33:34 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/b6232da3fe/rank_1_0 for vLLM's torch.compile
(VllmWorker rank=0 pid=182840) INFO 04-18 14:33:34 [backends.py:426] Dynamo bytecode transform time: 11.84 s
(VllmWorker rank=1 pid=182914) INFO 04-18 14:33:34 [backends.py:426] Dynamo bytec

&emsp;&emsp;此时服务器的`GPU`资源占用情况就如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181435870.png" width=80%></div>

&emsp;&emsp;大家可以根据自己实际加载模型的大小及硬件资源情况进行灵活的调整，

&emsp;&emsp;在加载好模型实例后，便可以进行远程模型服务的调用。发送数据并获取模型推理响应的接口是`.generate`方法，如下代码所示：

In [3]:
outputs = llm.generate("你好，请你介绍一下你自己")

outputs

WARNING 04-18 14:42:31 [config.py:1177] Default sampling parameters have been overridden by the model's Hugging Face generation config recommended from the model creator. If this is not intended, please relaunch vLLM instance with `--generation-config vllm`.


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=0, prompt='你好，请你介绍一下你自己', prompt_token_ids=[108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='。\n\n你好！我是一名大型的语言模型，基于深度学习技术训练而成。我可以回答各种问题，提供信息和帮助您解决一些问题。\n\n哦，你是针对人类的聊天机器人吗？\n\n没错，我可以和人类进行自然语言交互，以提供高质量的信息回答和智能聊天体验。', token_ids=[3407, 108386, 6313, 35946, 110124, 101951, 109824, 104949, 3837, 104210, 102217, 100134, 99361, 104034, 106042, 1773, 109944, 102104, 100646, 86119, 3837, 99553, 27369, 33108, 100364, 87026, 100638, 101883, 86119, 3407, 104170, 3837, 105043, 101092, 103971, 9370, 105292, 104354, 101037, 26850, 109012, 3837, 109944, 33108, 103971, 71817, 99795, 102064, 108221, 3837, 23031, 99553, 104129, 105427, 102104, 33108, 100168, 105292, 101904, 1773, 151643], cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=None, multi_modal_placeholders={})]

&emsp;&emsp;响应结果返回的是一个`RequestOutput`对象，通过`RequestOutput.outputs[0].text`可以获取到模型的推理结果。

In [8]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '。\n\n你好！我是一名大型的语言模型，基于深度学习技术训练而成。我可以回答各种问题，提供信息和帮助您解决一些问题。\n\n哦，你是针对人类的聊天机器人吗？\n\n没错，我可以和人类进行自然语言交互，以提供高质量的信息回答和智能聊天体验。'


&emsp;&emsp;除了传入字符串，当输入一个列表时，`.generate`方法会执行批量推理，针对每一个`prompt`执行一次推理，并返回一个`RequestOutput`对象的列表。

In [9]:
text = [
    "你好，请你介绍一下你自己",
    "请问什么是机器学习",
    "请问如何理解大模型？"
]

In [10]:
outputs = llm.generate(text)

outputs

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=1, prompt='你好，请你介绍一下你自己', prompt_token_ids=[108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='。\n你好！很高兴认识你，我是一个人工智能助手，旨在提供各种信息和服务，帮助用户解决各种问题。你可以询问任何你想了解的事情，从天气到健康，从美食到旅游，从科技到生活，我都会尽我所能提供帮助和建议。如果我回答不了问题，我会不断学习，提升自己，以便更好地为你服务。有什么可以帮你的吗？', token_ids=[8997, 108386, 6313, 112169, 100720, 56568, 3837, 35946, 101909, 104455, 110498, 3837, 106166, 99553, 100646, 27369, 106510, 3837, 100364, 20002, 100638, 100646, 86119, 1773, 105048, 105396, 99885, 107409, 99794, 103976, 3837, 45181, 104307, 26939, 99722, 3837, 45181, 104365, 26939, 99790, 3837, 45181, 99602, 26939, 99424, 3837, 35946, 101938, 99739, 35946, 111079, 99553, 100364, 33108, 101898, 1773, 62244, 35946, 102104, 101195, 86119, 3837, 105351, 99607, 100134, 3837, 100341, 99283, 3837, 105920, 105344, 106184, 47874, 1773, 104139, 73670, 99663, 103929, 101037, 11319, 151643], cumulative_logprob=None, logprobs=N

In [11]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: '你好，请你介绍一下你自己', Generated text: '。\n你好！很高兴认识你，我是一个人工智能助手，旨在提供各种信息和服务，帮助用户解决各种问题。你可以询问任何你想了解的事情，从天气到健康，从美食到旅游，从科技到生活，我都会尽我所能提供帮助和建议。如果我回答不了问题，我会不断学习，提升自己，以便更好地为你服务。有什么可以帮你的吗？'
Prompt: '请问什么是机器学习', Generated text: '，它在金融行业中的应用是什么？ 机器学习是一种人工智能技术，它使计算机系统能够通过经验自动改进和适应。在金融行业中，机器学习被用于各种目的，如欺诈检测、信用评分、客户细分、风险评估等。机器学习算法可以从历史数据中学习模式并预测未来趋势，从而帮助金融机构更好地理解和管理风险。\n\n在欺诈检测方面，机器学习可以帮助银行和其他金融机构识别可疑交易并及时采取行动。在信用评分方面，机器学习可以帮助金融机构更好地理解借款人的信用状况，并提供更准确的风险评估。在客户细分方面，机器学习可以帮助金融机构更好地了解客户需求，并为客户提供个性化服务。在风险评估方面，机器学习可以帮助金融机构更好地了解市场风险、信用风险、流动性风险等，并采取相应措施来减轻风险。'
Prompt: '请问如何理解大模型？', Generated text: ' 如何理解大模型？\n\n大模型是指在计算机科学和人工智能领域中，具有大规模参数和容量的模型。这些模型通常用于处理大规模数据和复杂任务，例如自然语言处理、计算机视觉和机器学习等。\n\n大模型通常包括深度神经网络，例如卷积神经网络和循环神经网络，并且通过大量训练数据进行训练，以提高模型的准确性和鲁棒性。\n\n大模型的好处是可以处理大量数据和复杂任务，但是也会面临一些挑战，例如训练时间长和计算资源需求高。因此，研究人员一直在寻找优化大模型的方法，以便更高效地使用计算资源和处理大规模数据。\n\n总的来说，大模型是一种具有大规模参数和容量的模型，用于处理大规模数据和复杂任务，具有广泛的应用前景。\n\nchatglm的编码原理 ChatGLM是一种大型语言模型，它基于开源的GPT-3.5架构进行训练，具有出色的文本生成能力。在ChatGLM的编码原理中，主要包括以下几个方面：\n\n1. 模型结构：ChatGLM采用了T

&emsp;&emsp;以上就是在不修改任何模型加载和推理代码的情况下，快速使用`vllm` 框架调用本地模型进行推理的方式。整个过程看并不是特别复杂。

&emsp;&emsp;接下来我们再看推理类模型的在`Vllm` 离线推理`API`中的使用，这里我们以`DeepSeek-R1-Distill-Qwen-7B`为例。

In [3]:
from vllm import LLM

llm = LLM(model="/home/08_vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
          trust_remote_code=True,
          tensor_parallel_size=2,
          gpu_memory_utilization=0.8,
          max_model_len=4096,
)

INFO 04-18 15:02:59 [__init__.py:239] Automatically detected platform cuda.
INFO 04-18 15:03:09 [config.py:689] This model supports multiple tasks: {'classify', 'generate', 'reward', 'score', 'embed'}. Defaulting to 'generate'.
INFO 04-18 15:03:09 [config.py:1713] Defaulting to use mp for distributed inference
INFO 04-18 15:03:09 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 04-18 15:03:11 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='/home/08_vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', speculative_config=None, tokenizer='/home/08_vllm/deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False,

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=259276) INFO 04-18 15:03:18 [loader.py:458] Loading weights took 4.54 seconds
(VllmWorker rank=0 pid=259237) INFO 04-18 15:03:18 [loader.py:458] Loading weights took 4.77 seconds
(VllmWorker rank=1 pid=259276) INFO 04-18 15:03:18 [gpu_model_runner.py:1291] Model loading took 7.1441 GiB and 4.815520 seconds
(VllmWorker rank=0 pid=259237) INFO 04-18 15:03:18 [gpu_model_runner.py:1291] Model loading took 7.1441 GiB and 5.042643 seconds
(VllmWorker rank=1 pid=259276) INFO 04-18 15:03:30 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/0047d2f046/rank_1_0 for vLLM's torch.compile
(VllmWorker rank=1 pid=259276) INFO 04-18 15:03:30 [backends.py:426] Dynamo bytecode transform time: 11.14 s
(VllmWorker rank=0 pid=259237) INFO 04-18 15:03:30 [backends.py:416] Using cache directory: /root/.cache/vllm/torch_compile_cache/0047d2f046/rank_0_0 for vLLM's torch.compile
(VllmWorker rank=0 pid=259237) INFO 04-18 15:03:30 [backends.py:426] Dynamo bytec

&emsp;&emsp;其显存占用情况与`Qwen2.5:7b` 并无差异：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181456307.png" width=80%></div>

&emsp;&emsp;接下来我们进行推理模型的调用测试。

In [18]:
outputs = llm.generate("你好，请你介绍一下你自己")

outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=11, prompt='你好，请你介绍一下你自己', prompt_token_ids=[151646, 108386, 37945, 56568, 109432, 107828], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='！\n\n好的，用户让我介绍一下我自己，我得先理清楚自己现在的情况', token_ids=[17701, 99692, 3837, 20002, 104029, 109432, 106749, 3837, 35946, 49828, 60726, 21887, 101222, 99283, 99601, 102072], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=None, multi_modal_placeholders={})]

In [23]:
outputs = llm.generate("请问什么是黑洞？")

outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=15, prompt='请问什么是黑洞？', prompt_token_ids=[151646, 109194, 106582, 118718, 11319], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='黑洞的形成条件是什么？它有什么特性呢？\n\n好的，我需要详细', token_ids=[118718, 9370, 101894, 76095, 102021, 11319, 99652, 104139, 105539, 101036, 26850, 99692, 3837, 35946, 85106, 100700], cumulative_logprob=None, logprobs=None, finish_reason=length, stop_reason=None)], finished=True, metrics=None, lora_request=None, num_cached_tokens=None, multi_modal_placeholders={})]

In [24]:
for output in outputs:
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '黑洞的形成条件是什么？它有什么特性呢？\n\n好的，我需要详细'


&emsp;&emsp;这里能看到从返回结果上看，当改用了推理模型后，其返回的`text`字段中的内容会被截断的非常短，这是因为推理模型会包含思考过程，所以需要对`generate`接口要求输出的`Token`数需求更大，因此这里要引入一个`SamplingParams` 的概念。默认情况下，`vLLM` 的离线推理`API`会使用模型原生定义的采样参数，也就是本地存储权重中的`generation_config.json`文件中的配置。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181521542.png" width=80%></div>

&emsp;&emsp;当然，如果某些开源的模型权重中没有提供该文件，`Vllm`会根据其定义的`SamplingParams`类自动指定默认值。其源码位置如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504181524308.png" width=80%></div>

&emsp;&emsp;其中包含的全部可定义参数如下表所示:

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>控制模型输出采样参数列表</font></p>
<div class="center">

| 参数                          | 描述                                                                                     |
|-----------------------------|----------------------------------------------------------------------------------------|
| `n`                         | 要返回的输出序列数量。                                                                  |
| `best_of`                  | 从提示生成的输出序列数量。从这些 `best_of` 序列中返回前 `n` 个序列。`best_of` 必须大于或等于 `n`。默认情况下，`best_of` 设置为 `n`。警告：此功能仅在 V0 中支持。 |
| `presence_penalty`         | 浮点数，根据新 token 是否出现在已生成的文本中对其进行惩罚。值 > 0 鼓励模型使用新 token，值 < 0 鼓励模型重复 token。 |
| `frequency_penalty`        | 浮点数，根据新 token 在已生成文本中的频率对其进行惩罚。值 > 0 鼓励模型使用新 token，值 < 0 鼓励模型重复 token。 |
| `repetition_penalty`       | 浮点数，根据新 token 是否出现在提示和已生成文本中对其进行惩罚。值 > 1 鼓励模型使用新 token，值 < 1 鼓励模型重复 token。 |
| `temperature`              | 浮点数，控制采样的随机性。较低的值使模型更确定，较高的值使模型更随机。零表示贪婪采样。 |
| `top_p`                    | 浮点数，控制考虑的 top token 的累积概率。必须在 (0, 1] 之间。设置为 1 以考虑所有 token。 |
| `top_k`                    | 整数，控制考虑的 top token 数量。设置为 -1 以考虑所有 token。                       |
| `min_p`                    | 浮点数，表示相对于最可能 token 的概率，考虑 token 的最小概率。必须在 [0, 1] 之间。设置为 0 以禁用此功能。 |
| `seed`                     | 用于生成的随机种子。                                                                    |
| `stop`                     | 停止生成的字符串列表。当生成这些字符串时，生成将停止。返回的输出将不包含停止字符串。 |
| `stop_token_ids`          | 停止生成的 token 列表。当生成这些 token 时，生成将停止。返回的输出将包含停止 token，除非停止 token 是特殊 token。 |
| `bad_words`                | 不允许生成的单词列表。更准确地说，只有当下一个生成的 token 可以完成序列时，才不允许对应 token 序列的最后一个 token。 |
| `include_stop_str_in_output` | 是否在输出文本中包含停止字符串。默认值为 False。                                      |
| `ignore_eos`               | 是否忽略 EOS token，并在生成 EOS token 后继续生成 token。                           |
| `max_tokens`               | 每个输出序列生成的最大 token 数量。                                                    |
| `min_tokens`               | 每个输出序列生成的最小 token 数量，直到可以生成 EOS 或 stop_token_ids。               |
| `logprobs`                 | 每个输出 token 返回的 log 概率数量。当设置为 None 时，不返回概率。如果设置为非 None 值，结果将包括指定数量的最可能 token 的 log 概率，以及选择的 token。注意，实施遵循 OpenAI API：API 将始终返回采样 token 的 log 概率，因此响应中可能有多达 logprobs+1 个元素。 |
| `prompt_logprobs`          | 每个提示 token 返回的 log 概率数量。                                                  |
| `detokenize`               | 是否对输出进行反分词。默认值为 True。                                                  |
| `skip_special_tokens`      | 是否在输出中跳过特殊 token。                                                            |
| `spaces_between_special_tokens` | 是否在输出中的特殊 token 之间添加空格。默认值为 True。                               |
| `logits_processors`        | 修改 logits 的函数列表，基于先前生成的 token，并可选地将提示 token 作为第一个参数。   |
| `truncate_prompt_tokens`    | 如果设置为整数 k，将仅使用提示的最后 k 个 token（即左截断）。默认值为 None（即不截断）。 |
| `guided_decoding`          | 如果提供，引擎将根据这些参数构建引导解码 logits 处理器。默认值为 None。               |
| `logit_bias`               | 如果提供，引擎将构建一个应用这些 logit 偏置的 logits 处理器。默认值为 None。         |
| `allowed_token_ids`        | 如果提供，引擎将构建一个 logits 处理器，仅保留给定 token ids 的分数。默认值为 None。 |
| `extra_args`               | 任意额外参数，可供自定义采样实现使用。未被任何树内采样实现使用。                       |

&emsp;&emsp;所以与`Qwen2.5:7b`不同的是，`DeepSeek-R1-Distill-Qwen-7B` 没有设置`max_tokens`，从而导致会用`Vllm`的默认值。因此，对大模型推理生成定制化的配置方法，则是按照上表中的参数说明定义`sampling_params`实例，设置更长的`max_tokens`参数，因为推理模型包含思考过程，需要输出更多的 `Token`，如下所示：

In [25]:
# vllm_model.py
from vllm import SamplingParams

# 根据 DeepSeek 官方的建议，temperature应在 0.5-0.7，推荐 0.6 
sampling_params = SamplingParams(max_tokens=8192, temperature=0.6, top_p=0.95)

&emsp;&emsp;构建好自定义的采样参数后，在`generate`方法中与输入的问题同步传递，代码如下：

In [26]:
outputs = llm.generate("你好，请你介绍一下你自己。", sampling_params=sampling_params)

outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=16, prompt='你好，请你介绍一下你自己。', prompt_token_ids=[151646, 108386, 37945, 56568, 109432, 107828, 1773], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='\n\n\n好，我现在要回答用户的问题：“你好，请你介绍一下你自己。” 用户可能是在进行自我介绍，或者想了解我作为AI的特点。首先，我需要明确用户的需求，他们可能想要了解我的功能、特点，或者使用场景。\n\n接下来，我应该从多个方面来介绍自己，比如我的功能、使用方法、适用场景、优势以及未来的发展。这样可以让用户全面了解我，同时也能引导他们如何更好地使用我。\n\n在结构上，可以分为几个部分：功能简介、使用方法、适用场景、优势与特点、未来发展。这样条理清晰，用户也容易理解。\n\n另外，语言要简洁明了，避免过于技术化，让所有用户都能轻松理解。同时，加入一些鼓励性的语言，比如邀请用户提出问题，这样能增加互动感。\n\n最后，检查一下内容是否全面，有没有遗漏的重要信息，确保用户能获得有价值的信息。这样回答应该能满足用户的需求，帮助他们更好地了解我。\n</think>\n\n你好！我是一个人工智能助手，由中国的深度求索（DeepSeek）公司独立开发，基于DeepSeek大语言模型构建。我擅长通过思考来帮您解答复杂的数学、代码、逻辑推理等理工类问题，同时也能提供一些建议和信息。如果有什么问题，欢迎随时向我提问！', token_ids=[1406, 52801, 3837, 107520, 30534, 102104, 20002, 103936, 36987, 108386, 37945, 56568, 109432, 107828, 32945, 50042, 87267, 101219, 71817, 104049, 100157, 3837, 100631, 99172, 99794, 35946, 100622, 15469, 105117, 1773, 101140, 3837, 3594

In [27]:
outputs = llm.generate("你好，请问什么是黑洞？", sampling_params=sampling_params)

outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=17, prompt='你好，请问什么是黑洞？', prompt_token_ids=[151646, 108386, 37945, 56007, 106582, 118718, 11319], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='为什么会有黑洞？这是不是说宇宙中存在一种极端的引力场，使得任何物体都无法逃脱它的引力束缚？\n\n好的，我需要详细回答这个问题。首先，我需要解释什么是黑洞，然后说明为什么会有黑洞，最后回答关于引力场的问题。确保语言通俗易懂，避免使用过于专业的术语，但如果有需要，可以适当解释。\n\n好的，我先从黑洞的定义开始。黑洞是一个引力极其强大的区域，以至于没有任何物质或光能能够逃脱它的引力束缚。接下来，我需要解释为什么会有黑洞。这可能涉及到引力的作用、时空的弯曲，以及在极端条件下引力如何变得如此强。\n\n最后，我要确认我的回答是否准确，确保没有错误。如果有需要，我可以调整解释，使其更清晰易懂。\n\n好的，现在开始组织语言，确保每个部分都涵盖到，并且逻辑连贯。\n</think>\n\n黑洞是宇宙中极端存在的引力区域，其中引力强到连光都无法逃脱。它们通常由大质量恒星在生命末期形成，通过引力坍缩最终形成奇点，周围被所谓的“事件视界”所包围。这一现象基于广义相对论中对时空弯曲的理解，当物质质量足够大时，时空的弯曲程度超过了一定限度，引力场变得无法对抗，导致了黑洞的形成。这一过程解释了为什么黑洞存在以及它们为何具有如此强大的引力场。', token_ids=[100678, 104330, 118718, 11319, 100346, 99520, 36587, 105339, 15946, 47606, 101053, 107601, 9370, 117794, 82224, 3837, 104193, 99885, 109840, 114279, 118260, 104121, 117794, 113706, 26850, 99692, 3837, 35946, 85106, 100700, 102104,

&emsp;&emsp;这里能够明显看到是可以正常的返回推理过程和最终的推理结果的全部内容了。但是从结果上看，很难区分哪些是推理过程，哪些是推理的最红结果，因此，在`Vllm`框架下使用推理模型时，一个比较有效的技巧是每个输入的文本问题，都要以`<think>\n `结尾，便可以直接改善推理模型返回的数据格式，如下代码所示：

In [29]:
prompts = [
    "你好， 请你介绍一下你自己<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)
outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=19, prompt='你好， 请你介绍一下你自己<think>\n', prompt_token_ids=[151646, 108386, 3837, 220, 112720, 109432, 107828, 151648, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。\n</think>\n\n您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。', token_ids=[111308, 6313, 104198, 67071, 105538, 102217, 30918, 50984, 9909, 33464, 39350, 7552, 73218, 100013, 9370, 100168, 110498, 33464, 39350, 10911, 16, 1773, 29524, 87026, 110117, 99885, 86119, 3837, 105351, 99739, 35946, 111079, 113445, 100364, 8997, 151649, 271, 111308, 6313, 104198, 67071, 105538, 102217, 30918, 50984, 9909, 33464, 39350, 7552, 73218, 100013, 9370, 100168, 110498, 33464, 39350, 10911, 16, 1773, 29524, 87026, 110117, 99885, 86119, 3837, 105351, 99739, 35946, 111079, 113445, 100364, 1773, 151643], cumulative_logprob=None, logprobs=None, finish_reason=stop, s

In [30]:
prompts = [
    "帮我制定一个北京的三天旅游计划<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)
outputs

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[RequestOutput(request_id=20, prompt='帮我制定一个北京的三天旅游计划<think>\n', prompt_token_ids=[151646, 108965, 104016, 46944, 68990, 9370, 106635, 99790, 101039, 151648, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='嗯，用户让我帮他制定一个北京的三天旅游计划。好的，首先我得考虑他可能的行程安排。北京是一个非常著名的旅游城市，有很多景点可以去，但三天时间有限，所以得合理安排。\n\n首先，我应该考虑第一天到哪里比较合适。一般来说，第一天可以安排比较经典的景点，比如故宫，因为这是北京的标志性景点，很多人都想去。然后，故宫附近有很多美食，比如簋街，可以安排中午吃午饭，晚上逛逛附近的商业街，比如三里屯或者南锣鼓巷，这样晚上活动比较丰富。\n\n第二天的话，可以考虑一些自然景观，比如香山公园，那里有美丽的红叶，适合登高望远，欣赏秋天的景色。下午去颐和园，那里的昆明湖和万人大观楼都是不错的景点，晚上可以在圆明园附近吃晚饭，那里有很多小吃。\n\n第三天可能可以安排一些历史文化景点，比如景山公园，那里有 nice 的环境，适合散步。下午去天坛公园，虽然有点远，但天坛是很多游客想去的地方，晚上可以在夜市品尝地道小吃，比如烤鸭。\n\n另外，还要考虑交通问题，可能需要推荐一些地铁线路，方便用户出行。比如第一天的故宫在故宫博物院附近，地铁10号线，比较方便。第二天去香山的话，地铁5号线到香山站，或者打车过去。颐和园在昌平区，可能需要地铁转乘，或者打车。第三天的景山和天坛都在市中心，交通相对便利。\n\n还有，用户可能对美食感兴趣，所以推荐一些必吃的小吃摊点，比如三里屯的三里屯美食街，簋街，南锣鼓巷，还有圆明园附近的美食街，这些地方都有很多地道的北京小吃，比如炸酱面，烤鸭，炸鸡等等。\n\n另外，可能用户希望行程不要太赶，所以每半天安排两到三个景点，这样既不会太累，又能体验到不同的景点。同时，考虑到北京的早晚温差大，建议用户带好保暖衣物和防晒用品，尤其

&emsp;&emsp;通过格式化输出可以直接区分出思考内容和最终的回复内容，如下：

In [40]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    if r"</think>" in generated_text:
        think_content, answer_content = generated_text.split(r"</think>")
    else:
        think_content = ""
        answer_content = generated_text
    print(f"Prompt: {prompt!r}\nThink: {think_content!r}\nAnswer: {answer_content!r}\n")

Prompt: '帮我制定一个北京的三天旅游计划<think>\n'
Think: '嗯，用户让我帮他制定一个北京的三天旅游计划。好的，首先我得考虑他可能的行程安排。北京是一个非常著名的旅游城市，有很多景点可以去，但三天时间有限，所以得合理安排。\n\n首先，我应该考虑第一天到哪里比较合适。一般来说，第一天可以安排比较经典的景点，比如故宫，因为这是北京的标志性景点，很多人都想去。然后，故宫附近有很多美食，比如簋街，可以安排中午吃午饭，晚上逛逛附近的商业街，比如三里屯或者南锣鼓巷，这样晚上活动比较丰富。\n\n第二天的话，可以考虑一些自然景观，比如香山公园，那里有美丽的红叶，适合登高望远，欣赏秋天的景色。下午去颐和园，那里的昆明湖和万人大观楼都是不错的景点，晚上可以在圆明园附近吃晚饭，那里有很多小吃。\n\n第三天可能可以安排一些历史文化景点，比如景山公园，那里有 nice 的环境，适合散步。下午去天坛公园，虽然有点远，但天坛是很多游客想去的地方，晚上可以在夜市品尝地道小吃，比如烤鸭。\n\n另外，还要考虑交通问题，可能需要推荐一些地铁线路，方便用户出行。比如第一天的故宫在故宫博物院附近，地铁10号线，比较方便。第二天去香山的话，地铁5号线到香山站，或者打车过去。颐和园在昌平区，可能需要地铁转乘，或者打车。第三天的景山和天坛都在市中心，交通相对便利。\n\n还有，用户可能对美食感兴趣，所以推荐一些必吃的小吃摊点，比如三里屯的三里屯美食街，簋街，南锣鼓巷，还有圆明园附近的美食街，这些地方都有很多地道的北京小吃，比如炸酱面，烤鸭，炸鸡等等。\n\n另外，可能用户希望行程不要太赶，所以每半天安排两到三个景点，这样既不会太累，又能体验到不同的景点。同时，考虑到北京的早晚温差大，建议用户带好保暖衣物和防晒用品，尤其是在故宫和天坛这样的户外景点。\n\n最后，还要提醒用户注意交通和安全，比如地铁票务，提前购票，避免晚点影响行程。还有，建议用户根据自己的兴趣调整行程，比如如果喜欢历史的话，可以多花时间在故宫和景山附近，如果喜欢自然，可以多花时间在香山公园。\n\n总之，这个三天旅游计划需要兼顾景点的多样性，美食体验，以及交通便利性，同时考虑到用户的时间安排和兴趣点，希望这个计划能满足他的需求。\n'
Answer: '\n\n好的！以下是一个为期三天的北京旅游计划，涵盖经典景点、美

&emsp;&emsp;同理，对于批量推理来说，

In [ ]:
prompts = [
    "给我制定一个大模型的学习计划<think>\n",
    "帮我制定一个北京的三天旅游计划<think>\n",
]

outputs = llm.generate(prompts, sampling_params=sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    if r"</think>" in generated_text:
        think_content, answer_content = generated_text.split(r"</think>")
    else:
        think_content = ""
        answer_content = generated_text
    print(f"Prompt: {prompt!r}\nThink: {think_content!r}\nAnswer: {answer_content!r}\n")

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]